# Odległość edycyjna

Zadanie dotyczy wykorzystania odległości edycyjnej.
1. Zaimplementuj algorytm obliczania odległości edycyjnej w taki sposób, aby możliwe było określenie przynajmniej jednej sekwencji edycji (dodanie, usunięcie,
zmiana znaku), która pozwala w minimalnej liczbie kroków, przekształcić jeden łańcuch w drugi.
2. Na podstawie poprzedniego punktu zaimplementuj prostą wizualizację działania algorytmu, poprzez wskazanie kolejnych wersji pierwszego łańcucha, w których
dokonywana jest określona zmiana. "Wizualizacja" może działać w trybie tekstowym. Np. zmiana łańcuch "los” w "kloc” może być zrealizowana następująco:
    1. \*k\*los (dodanie litery k)
    2. klo\*c\* (zamiana s->C)
3. Przedstaw "nik działania algorytmu z p. 2 dla następujących par łańcuchów:
    1. los - kloc
    2. Łódź - Lodz
    3. kwintesencja - quintessence
    4. ATGAATCTTACCGCCTCG - ATGAGGCTCTGGCCCCTG
4. Zaimplementuj algorytm obliczania najdłuższego wspólnego podciągu dla pary ciągów elementów.
5. Korzystając z gotowego tokenizera (np spacy - https://spacy.io/api/tokenizer) dokonaj podziału załączonego tekstu na tokeny.
6. Stwórz 2 wersje załączonego tekstu, w których usunięto 3% losowych tokenów.
7. Oblicz długość najdłuższego podciągu wspólnych tokenów dla tych tekstów. Traktujemy teraz token (wyraz) jako podstawową, niepodzielną jednostkę ciągu.
8. Korzystając z algorytmu z punktu 4 skonstruuj narzędzie, o działaniu podobnym do narzędzia diff, tzn. wskazującego w dwóch plikach linie, które się różnią. Na
Ujściu narzędzia powinny znaleźć się elementy, które nie należą do najdłuższego wspólnego podciągu. Należy wskazać skąd dana linia pochodzi (< > -
pierwszy/drugi plik) oraz numer linii w danym pliku. Traktujemy teraz całą linię jako podstawową, niepodzielną jednostkę ciągu.

    **(tutaj zamiast "< >" użyłem "- +")**

9. Przedstaw wynik działania narzędzia na tekstach z punktu 6. Zwróć uwagę na dodanie znaków przejścia do nowej linii, które są usuwane w trakcie tokenizacji.

In [159]:
import numpy as np
import colorama
from spacy.lang.pl import Polish
import random

In [140]:
''' Edit distance between two strings '''

def levenshtein_matrix(str1, str2):
    matrix = np.empty((len(str1)+1, len(str2)+1), dtype=int)
    matrix[0, :] = np.arange(len(str2)+1)
    matrix[:, 0] = np.arange(len(str1)+1)
    for i in range(1, len(str1)+1):
        for j in range(1, len(str2)+1):
            matrix[i, j] = min(matrix[i-1, j]+1, matrix[i, j-1]+1, matrix[i-1, j-1]+(str1[i-1]!=str2[j-1]))
    return matrix

def levenshtein_distance(str1, str2):
    return levenshtein_matrix(str1, str2)[-1, -1]

def levenshtein_bactrace(str1, str2, trace_print=True):
    if trace_print:
        print(">> TRACEBACK <<")
        print("Input string:", str1)
        print("Output string:", str2)
        print("--------------------")

    matrix = levenshtein_matrix(str1, str2)
    i, j = len(str1), len(str2)
    trace = [(i, j, "end")]
    while i > 0 and j > 0:
        action = True
        if matrix[i, j] == matrix[i-1, j-1] + 1: # replace
            trace.append((i-1, j-1, "r"))
            input_str = str1[:i-1] + colorama.Fore.CYAN + str1[i-1] + colorama.Fore.RESET + str1[i:]
            output_str = str1[:i-1] + str2[j-1] + str1[i:]
            i, j = i-1, j-1
        elif matrix[i, j] == matrix[i-1, j] + 1: # delete
            trace.append((i-1, j, "d"))
            input_str = str1[:i-1] + colorama.Fore.RED + str1[i-1] + colorama.Fore.RESET + str1[i:]
            output_str = str1[:i-1] + str1[i:]
            i -= 1
        elif matrix[i, j] == matrix[i, j-1] + 1: # insert
            trace.append((i, j-1, "i"))
            input_str = str1[:i] + colorama.Fore.GREEN + str2[j-1] + colorama.Fore.RESET + str1[i:]
            output_str = str1[:i] + str2[j-1] + str1[i:]
            j -= 1
        else: # match
            action = False
            trace.append((i-1, j-1, "m"))
            input_str = output_str = str1
            i, j = i-1, j-1

        if trace_print and action:
            print(input_str, output_str, sep=" -> ")
        str1 = output_str

    while i > 0:
        trace.append((i-1, j, "d"))
        if trace_print:
            input_str = str1[:i-1] + colorama.Fore.RED + str1[i-1] + colorama.Fore.RESET + str1[i:]
            output_str = str1[:i-1] + str1[i:]
            print(input_str, output_str, sep=" -> ")
        str1 = output_str
        i -= 1

    while j > 0:
        trace.append((i, j-1, "i"))
        if trace_print:
            input_str = str1[:i] + colorama.Fore.GREEN + str2[j-1] + colorama.Fore.RESET + str1[i:]
            output_str = str1[:i] + str2[j-1] + str1[i:]
            print(input_str, output_str, sep=" -> ")
        str1 = output_str
        j -= 1
    
    if trace_print:
        
        print("--------------------")

    return trace


In [141]:
levenshtein_bactrace("los", "kloc");

>> TRACEBACK <<
Input string: los
Output string: kloc
--------------------
los -> loc
kloc -> kloc
--------------------


In [142]:
levenshtein_bactrace("Łódź", "Lodz");

>> TRACEBACK <<
Input string: Łódź
Output string: Lodz
--------------------
Łódź -> Łódz
Łódz -> Łodz
Łodz -> Lodz
--------------------


In [143]:
levenshtein_bactrace("kwintesencja", "quintessence");

>> TRACEBACK <<
Input string: kwintesencja
Output string: quintessence
--------------------
kwintesencja -> kwintesencje
kwintesencje -> kwintesence
kwintessence -> kwintessence
kwintessence -> kuintessence
kuintessence -> quintessence
--------------------


In [144]:
levenshtein_bactrace("ATGAATCTTACCGCCTCG", "ATGAGGCTCTGGCCCCTG");

>> TRACEBACK <<
Input string: ATGAATCTTACCGCCTCG
Output string: ATGAGGCTCTGGCCCCTG
--------------------
ATGAATCTTACCGCCTCG -> ATGAATCTTACCGCCTG
ATGAATCTTACCGCCTG -> ATGAATCTTACCCCCTG
ATGAATCTTACCCCCTG -> ATGAATCTTAGCCCCTG
ATGAATCTTAGCCCCTG -> ATGAATCTTGGCCCCTG
ATGAATCTCTGGCCCCTG -> ATGAATCTCTGGCCCCTG
ATGAATCTCTGGCCCCTG -> ATGAAGCTCTGGCCCCTG
ATGAAGCTCTGGCCCCTG -> ATGAGGCTCTGGCCCCTG
--------------------


In [203]:
''' Longest common subsequence '''

def lcs_matrix(str1, str2):
    matrix = np.empty((len(str1)+1, len(str2)+1), dtype=int)
    matrix[0, :] = np.zeros(len(str2)+1)
    matrix[:, 0] = np.zeros(len(str1)+1)
    for i in range(1, len(str1)+1):
        for j in range(1, len(str2)+1):
            if str1[i-1] == str2[j-1]:
                matrix[i, j] = matrix[i-1, j-1] + 1
            else:
                matrix[i, j] = max(matrix[i-1, j], matrix[i, j-1])
    return matrix

def lcs(str1, str2):
    matrix = lcs_matrix(str1, str2)
    i, j = len(str1), len(str2)
    lcs = []
    while i > 0 and j > 0:
        if str1[i-1] == str2[j-1]:
            lcs.append(str1[i-1])
            i, j = i-1, j-1
        elif matrix[i-1, j] > matrix[i, j-1]:
            i -= 1
        else:
            j -= 1

    while i > 0:
        lcs.append(str1[i-1])
        i -= 1

    while j > 0:
        lcs.append(str2[j-1])
        j -= 1

    return ''.join(reversed(lcs))


In [157]:
''' Tokenizer '''

nlp = Polish()
def tokenize(text):
    return [token.text for token in nlp.tokenizer(text)]

In [252]:
''' LCS on tokens '''

f = open("romeo-i-julia-700.txt", "r", encoding="utf-8")
text = f.read()
f.close()

tokens = tokenize(text)

def delete_random_tokens(tokens, n):
    for _ in range(n):
        tokens.pop(random.randint(0, len(tokens)-1))
    return tokens

n = round(len(tokens) * 0.03)
tokens1 = delete_random_tokens(tokens.copy(), n)
tokens2 = delete_random_tokens(tokens.copy(), n)

with open("output1.txt", "w", encoding="utf-8") as f1, open("output2.txt", "w", encoding="utf-8") as f2:
    f1.write(' '.join(tokens1))
    f2.write(' '.join(tokens2))


In [254]:
print("LCS length:", len(lcs(tokens1, tokens2)))

LCS length: 10450


In [259]:
''' diff '''

def compare_files(file1, file2):
    with open(file1, 'r', encoding='utf-8') as f1, open(file2, 'r', encoding='utf-8') as f2:
        lines1 = f1.readlines()
        lines2 = f2.readlines()

        lcs_lines = lcs(lines1, lines2)

        f1_count = len([line for line in lines1 if line not in lcs_lines])
        f2_count = len([line for line in lines2 if line not in lcs_lines])

        print(">> FILE DIFFERENCES <<")
        print("---", file1)
        print("+++", file2)
        print(f"@@ -{f1_count} | +{f2_count} @@")
        print("----------------------\n")
        
        i, j = 0, 0
        while i < len(lines1) and j < len(lines2):
            if lines1[i] in lcs_lines:
                i += 1
            elif lines2[j] in lcs_lines:
                j += 1
            else:
                if i < j:
                    print("-", i+1, colorama.Fore.RED + lines1[i] + colorama.Fore.RESET, end="")
                    i += 1
                else:
                    print("+", j+1, colorama.Fore.GREEN + lines2[j] + colorama.Fore.RESET, end="")
                    j += 1
                

In [258]:
compare_files("output1.txt", "output2.txt")

>> FILE DIFFERENCES <<
--- output1.txt
+++ output2.txt
@@ -123 | +127 @@
----------------------

+ 16   * MERKUCJO — krewny księcia * BENWOLIO — synowiec Montekiego 
- 16   * MERKUCJO — krewny księcia 
- 17   * BENWOLIO — synowiec Montekiego 
+ 30   * JULIA — córka Kapuletów 
+ 31   MARTA — mamka Julii 
- 31   JULIA — córka Kapuletów 
+ 32   * Obywatele , różne osoby płci obojej , liczący się do przyjaciół obu domów , maski , straż wojskowa i inne osoby . 
- 32   * MARTA — mamka Julii 
- 33   * Obywatele weroneńscy , różne osoby płci obojej , liczący się do przyjaciół obu domów , maski , straż wojskowa i inne osoby . 
+ 45  Dwa , zacne jednako i sławne — 
+ 46  Tam , gdzie się rzecz ta rozgrywa , w Weronie , 
- 46  Dwa rody , zacne jednako i sławne — 
+ 47  Do nowej zbrodni pchają złości dawne , 
- 47  Tam , gdzie się rzecz ta , w Weronie , Do nowej zbrodni pchają złości dawne , 
+ 50  Z łon tych dwu wzięło bowiem życie , 
- 50  Z łon tych dwu wrogów wzięło bowiem życie , 
+ 52  Po peł